# Joint Intent Classification and Slot Filliing with Transformers

Code get and modified from the Master Datascience Paris Saclay. [here](https://github.com/m2dsupsdlclass/lectures-labs).

**Goal**
* Fine-tune a pretrained transformer-based neural network model to convert a user qeury expressed in English into a representation that is structured enough to be processed by an automated service.

Here is an example of interpretation computed by such a Natural Language Understanding system:
    
    >>> nlu('Book a table for two at Le Ritz for Friday night",
            tokenizer, joint_model, intent_names, slot_names)
    {
        'intent': 'BookRestaurant',
        'slots': {
            'party_size_number': 'two',
            'restaurant_name': 'Le Ritz',
            'timeRange': 'Friday night'
        }
    }
    
Intent classification is a simple classification problem. The trick is to treat the structured knowledge extraction part ("Slot Filling") as a token-level classification problem using BIO-annotations:

    >>> show_predictions('Book a table for two at Le Ritz for Friday night',
                         tokenizer, joint_model, intent_names, slot_names)
    ## Intent: BookRestaurant
    ## Slots:
      Book : O
         a : O
     table : O
       for : O
       two : B-party_size_number
        at : O
        Le : B-restaurant_name
         R : I-restaurant_name
     ##itz : I-restaurant_name
       for : O
    Friday : B-timeRange
     night : I-timeRange
     
We will show hhow to train a such "sequence classification" and "token classification" joint model on a voice command dataset published by snips.ai. This notebook is a partial reproduction of some of the results presented in this paper: BERT for Joint Intent Classification and Shot Filling, Qian Chen, Zhu Zhuo, Wen Wang [link](https://arxiv.org/abs/1902.10909).

In [ ]:
# Load packages

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from transformers import BertTokenizer, TFBertModel,pipeline,AutoTokenizer
from urllib.request import urlretrieve

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub

import datetime


In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
similarModel = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return similarModel(input)
test_embed = embed(["hello world"])

In [ ]:
SNIPS_DATA_BASE_URL = (
    "https://github.com/ogrisel/slot_filling_and_intent_detection_of_SLU/blob/"
    "master/data/MIT_corpus/restaurant/"
)
for filename in ["train", "valid", "test", "vocab.intent", "vocab.slot"]:
    path = Path(filename)
    if not path.exists():
        print(f"Downloading {filename}...")
        urlretrieve(SNIPS_DATA_BASE_URL + filename + "?raw=true", path)

Let's have a look at the first lines from the training set.

In [ ]:
lines_train = Path('train').read_text('utf-8').strip().splitlines()

In [ ]:
print(f'First line of training set: {lines_train[0]}.')

Some remarks:
* The class label for the voice command appears at the end of each line (after the "<=>" marker).
* Each word-level token is annotated with B-I-O labels using the "." separator.
* B-I-O stands for Beginning-Inside-Outside
* "Add:O" means that the token "Add" is "Outside" of any annotation span.
* "Don:B-entity_name" means that "Don" is the "Beginning" of an annotation of type "entity_name".
* "and:I-entity_name" means that "and" is "inside" the previously started annotation of type "entity_name".

Let's write a parsing function and test it on the first line.

In [ ]:
def parse_line(line):
    utterance_data, intent_label = line.split(" <=> ")
    items = utterance_data.split()
    words = [item.rsplit(':', 1)[0] for item in items]
    word_labels = [item.rsplit(':', 1)[1] for item in items]
    return {
        'intent_label': intent_label,
        'words': " ".join(words),
        'words_label': " ".join(word_labels),
        'length': len(words)
    }

In [ ]:
parse_line(lines_train[0])

In [ ]:
print(Path('vocab.intent').read_text('utf-8'))

In [ ]:
print(Path('vocab.slot').read_text('utf-8'))

In [ ]:
parsed = [parse_line(line) for line in lines_train]
df_train = pd.DataFrame([p for p in parsed if p is not None])

In [ ]:
# Print some lines of the training set
df_train.head(5)

In [ ]:
# Count the number of lines by intent label
df_train.intent_label.value_counts()

In [ ]:
# Histogram of sentence lengths
df_train.hist('length', bins=30)

In [ ]:
# Get validation and test set
lines_validation = Path('valid').read_text('utf-8').strip().splitlines()
lines_test = Path('test').read_text('utf-8').strip().splitlines()

df_validation = pd.DataFrame([parse_line(line) for line in lines_validation])
df_test = pd.DataFrame([parse_line(line) for line in lines_test])

## Intent classification (sentence level)

Let's ignore the slot filling task for now and let's try to build a sentence level classifier by fine-tuning a pre-trained Transformer-based model using the `huggingface/transformers` package that provides both Tensorflow/Keras and Pytorch APIs.

### The BERT tokenizer

First let's load a pre-trained tokenizer and test it on a test sentence from the training set.

In [ ]:
model_name = 'google/mobilebert-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
first_sentence = df_train.iloc[0]['words']
print(first_sentence)

In [ ]:
tokenizer.tokenize(first_sentence)

Notive that BERT uses subword tokens. So the length of the tokenized sentence is likely to be larger than the number of words in the sentence. It is particularly interesting to use subword tokenization sentence for general purpose language models such as BERT because it should be possible to generalize the model and then to fine-tuned it to be a specialized one.

Each token string is mapped to a unique integer id that makes it fast to lookup the right column in the input layer token embedding.

In [ ]:
# Encode sentence to id
tokenizer.encode(first_sentence)

In [ ]:
# Do the inverse operation
tokenizer.decode(tokenizer.encode(first_sentence))

Remarks:
* The first token `[CLS]` is used by the pre-training task for sequence classification.
* The last token `[SEP]` is a separator for the pre-training task that classifies if a pair of sentences are consecutive in a corpus or not (next sentence prediction).
* Here, we want to use BERT to compute a representation of a single voice command at a time.
* We could reuse the representation of the `[CLS]` token for sequence classification.
* Alternatively, we can pool the representations of all the tokens of the voice command (*e.g.* global average) and use that as the input of the final sequence classification layer.

In [ ]:
train_sequence_lengths = [len(tokenizer.encode(text))
                          for text in df_train['words']]
plt.hist(train_sequence_lengths, bins=30)
plt.title(f'Max sequence length: {max(train_sequence_lengths)}')
plt.xlabel('Length')
plt.ylabel('Count')
plt.show()

To perform transfer learning, we will need to work with padded sequences. So, they all have the same sizes. The above histograms, shows that after tokenization, $43$ tokens are enough to represent all the voice commands in the training set.

The mapping can be introspected in the `tokenizer.vocab` attribute.

In [ ]:
print(f'Vocabulary size: {tokenizer.vocab_size} words.')

In [ ]:
# Get the items in BERT
bert_vocab_items = list(tokenizer.vocab.items())

In [ ]:
# Print some examples of items
bert_vocab_items[250:260]

In [ ]:
def encode_dataset(tokenizer, text_sequences, max_length):
    token_ids = np.zeros(shape=(len(text_sequences), max_length),
                         dtype=np.int32)
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded
    attention_masks = (token_ids != 0).astype(np.int32)
    
    return {'input_ids': token_ids, 'attention_masks': attention_masks}

In [ ]:
encoded_train = encode_dataset(tokenizer, df_train['words'], 45)
encoded_validation = encode_dataset(tokenizer, df_validation['words'], 45)
encoded_test = encode_dataset(tokenizer, df_test['words'], 45)

In [ ]:
encoded_train['input_ids']

In [ ]:
encoded_train['attention_masks']

In [ ]:
intent_names = Path('vocab.intent').read_text('utf-8').split()
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))

In [ ]:
intent_map

In [ ]:
intent_train = df_train['intent_label'].map(intent_map).values
intent_validation = df_validation['intent_label'].map(intent_map).values
intent_test = df_test['intent_label'].map(intent_map).values

In [ ]:
base_bert_model = TFBertModel.from_pretrained('google/mobilebert-uncased')
base_bert_model.summary()

In [ ]:
outputs = base_bert_model(encoded_validation)

The first output of the BERT model is a tensor with shape: `(batch_size, seq_len, output_dim)` which computes features for each token in the input sequence.

In [ ]:
print(f'Shape of the first output of the BERT model: {outputs[0].shape}.')

The second output of the BERT model is a tensor with shape `(batch_size, output_dim)` which is the vector representation of the special token `[CLS]`. This vector is typically used as a pooled representation for the sequence as a whole. This will be used as the features of our latent classifier.

In [ ]:
print(f'Shape of the second output of the BERT model: {outputs[1].shape}.')

Let's build an train a sequece classification model using to predict the intent class. We will use the `self.bert` pretrained model in the `call` method and only consider the pooled features (ignore the token-wise features for now).

In [ ]:
# Define IntentClassification model
class IntentClassificationModel(tf.keras.Model):
    def __init__(self, intent_num_labels=None,
                 model_name='google/mobilebert-uncased',
                 dropout_prob=0.1):
        super().__init__(name='joint_intent_slot')
        # Let's preload the pretrained model BERT in the constructor
        # of our classifier model.
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        
        # Define a (Dense) classification layer to compute for each
        # sequence in a batch of samples. The number of output classes
        # is given by the intent_num_labels parameter.
        # Use the default linear activation (no softmax) to compute
        # logits. The softmax normalization will be computed in the
        # loss function instead of the model itself.
        self.intent_classifier = Dense(intent_num_labels)
        
    def call(self, inputs, **kwargs):
        # Use the pretrained model to extract features from our
        # encoded inputs.
        sequence_output, pooled_output = self.bert(inputs, **kwargs)
        
        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim) and gives a "pooled" representation
        # for the full sequence from the hidden state that corresponds
        # to the "[CLS]" token.
        pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
        
        # Use the classifier layer to compute the logits from the
        # pooled features.
        intent_logits = self.intent_classifier(pooled_output)
        return intent_logits

In [ ]:
%%timeit
# Build the model
intent_model = IntentClassificationModel(intent_num_labels=len(intent_map))

intent_model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08),
                     loss=SparseCategoricalCrossentropy(from_logits=True),
                     metrics=[SparseCategoricalAccuracy('accuracy')])

In [ ]:
# Train the model
#history = intent_model.fit(encoded_train, intent_train,
#                           epochs=2, batch_size=32,
#                           validation_data=(encoded_validation, intent_validation))

In [ ]:
def classify(text, tokenizerzer, model, intent_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :] # Batch size = 1
    class_id = model(inputs).numpy().argmax(axis=1)[0]
    return intent_names[class_id]

In [ ]:
# Example of classification
#classify('Which restaurant you would recommend',
   #      tokenizer, intent_model, intent_names)

In [ ]:
slot_names = ["[PAD]"]
slot_names += Path('vocab.slot').read_text('utf-8').strip().splitlines()

slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

In [ ]:
def encode_token_labels(text_sequences, slot_names, tokenizer, slot_map, max_length):
    encoded = np.zeros(shape=(len(text_sequences), max_length), dtype=np.int32)
    for i, (text_sequence, word_labels) in enumerate(
            zip(text_sequences, slot_names)):
        encoded_labels = []
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded

In [ ]:
slot_train = encode_token_labels(df_train['words'], df_train['words_label'], tokenizer, slot_map, 45)
slot_validation = encode_token_labels(df_validation['words'], df_validation['words_label'], tokenizer, slot_map, 45)
slot_test = encode_token_labels(df_test['words'], df_test['words_label'], tokenizer, slot_map, 45)

In [ ]:
# Define JointIntentAndSlotFilling model
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                 model_name="google/mobilebert-uncased", dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels,
                                       name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")

    def call(self, inputs, **kwargs):
        sequence_output, pooled_output = self.bert(inputs, **kwargs)

        # The first output of the main BERT layer has shape:
        # (batch_size, max_length, output_dim)
        sequence_output = self.dropout(sequence_output,
                                       training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim)
        # and gives a "pooled" representation for the full sequence from the
        # hidden state that corresponds to the "[CLS]" token.
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits

In [ ]:

joint_model = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

# Define one classification loss for each output:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]
metrics = [SparseCategoricalAccuracy('accuracy')]
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

In [ ]:

history = joint_model.fit(
    encoded_train, (slot_train, intent_train),
    validation_data=(encoded_validation, (slot_validation, intent_validation)),
    epochs=2, batch_size=32)

In [ ]:
joint_model.summary()

In [ ]:
def show_predictions(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    print("## Intent:", intent_names[intent_id])
    print("## Slots:")
    for token, slot_id in zip(tokenizer.tokenize(text), slot_ids):
        print(f"{token:>10} : {slot_names[slot_id]}")

In [ ]:
# Example of classification
show_predictions('Will it snow tomorrow in Paris?',
                 tokenizer, joint_model, intent_names, slot_names)

## Decoding predictions into structured knowledge

For completeness, here a minimal functional to naively decode the predicted BIO slot ids and convert it into a structured representation for the detected slots as a Python dictionaries.

In [ ]:
def decode_predictions(text, tokenizer, intent_names, slot_names,
                       intent_id, slot_ids):
    info = {"intent": intent_names[intent_id]}
    collected_slots = {}
    active_slot_words = []
    active_slot_name = None
    for word in text.split():
        tokens = tokenizer.tokenize(word)
        current_word_slot_ids = slot_ids[:len(tokens)]
        slot_ids = slot_ids[len(tokens):]
        current_word_slot_name = slot_names[current_word_slot_ids[0]]
        if current_word_slot_name == "O":
            if active_slot_name:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = []
                active_slot_name = None
        else:
            # Naive BIO: handling: treat B- and I- the same...
            new_slot_name = current_word_slot_name[2:]
            if active_slot_name is None:
                active_slot_words.append(word)
                active_slot_name = new_slot_name
            elif new_slot_name == active_slot_name:
                active_slot_words.append(word)
            else:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = [word]
                active_slot_name = new_slot_name
    if active_slot_name:
        collected_slots[active_slot_name] = " ".join(active_slot_words)
    info["slots"] = collected_slots
    return info

In [ ]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    return decode_predictions(text, tokenizer, intent_names, slot_names,
                              intent_id, slot_ids)

In [ ]:
nlu('I want some chinese food',
                 tokenizer, joint_model, intent_names, slot_names)

In [ ]:
nlu('tokyo restaurant in the north of town',
    tokenizer, joint_model, intent_names, slot_names)

## Pre Handling the Input from DST2


Get a glance at the dataset, output is what agent speak while input is what human speak


In [ ]:
dst2csv = '../input/modified-dst2/data.csv'
dst2data = pd.read_csv(dst2csv)

dst2data.head(5)

In [ ]:
sentiment_model = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_classifier = pipeline('sentiment-analysis',model=sentiment_model,tokenizer=AutoTokenizer.from_pretrained(sentiment_model))


In [ ]:
import json
import copy

import math

from scipy import spatial



def cosine_similarity(a,b):
    cosine_similarity = 1 - float(spatial.distance.cosine(a, b))
    return cosine_similarity


def countDictValue(dict1):
    count = 0
    for key,value in dict1.items():
        if type(value) == list:
            count+=len(value)
        else :
            count +=1
    return count


def getSentimentPoints(sentense):
    
    result = sentiment_classifier(sentense)[0]
    if("1" in result["label"]):
        return 0
    elif("0" in result["label"]):
        return -1
    else:
        return 1

def mergeDict(dict1,dict2):
    newDict = dict1
    for key,value in dict2.items():
        if key not in newDict:
            newDict[key] = value
        else :
            exist_value = newDict[key]
            # exist value can be list or normal variable
            if type(exist_value) == list:
                if value in exist_value:
                    pass
                else:
                    newDict[key].append(value)
            elif exist_value != value:
                newDict[key]=[exist_value,value]
    return newDict
    


    

class dialog :
    def __init__(self,id,input,output):
        self.input = input
        self.output = output
        self.id = id
        self.buildIndex()
        self.calculateSentiment()
        self.calculateSlots()
    
    # compare with previous 2 sentenses    
    def getSimilarity(self,index):
        input_similar_1 = 0
        output_similar_1 = 0
        input_similar_2 = 0
        output_similar_2 = 0
        output_vect = self.round_vector[index][0]
        input_vect = self.round_vector[index][1]
        if(index > 0):
            output_vect_1 = self.round_vector[index-1][0]
            input_vect_1 = self.round_vector[index-1][1]
            input_similar_1 = cosine_similarity(input_vect,input_vect_1)
            output_similar_1 = cosine_similarity(output_vect,output_vect_1)
        if(index>1):
            output_vect_2 = self.round_vector[index-2][0]
            input_vect_2 = self.round_vector[index-2][1]
            input_similar_2 = cosine_similarity(input_vect,input_vect_2)
            output_similar_2 = cosine_similarity(output_vect,output_vect_2)
        return input_similar_1,output_similar_1,input_similar_2,output_similar_2
    
    def buildIndex(self):
        output_sentenses = self.output.split("|")
        input_sentenses = self.input.split("|")
        self.round = {}
        self.round_vector = {}
        outside = 0
        inside = 0
        for sent in output_sentenses:
            if sent !="":
                outside+=1
        for sent in input_sentenses:
            if sent !="":
                inside+=1
        self.round_counter = min(outside,inside)
        for i in range(self.round_counter):
            if(output_sentenses[i]):
                self.round[i]=(output_sentenses[i],input_sentenses[i])
                embed_vector = embed([output_sentenses[i],input_sentenses[i]])
                self.round_vector[i] = (embed_vector[0].numpy(),embed_vector[1].numpy())

    def calculateSentiment(self):
        self.sentiment_list=[]
        for index in range(self.round_counter):
            output_sent = self.round[index][0]
            input_sent = self.round[index][1]
            self.sentiment_list.append((getSentimentPoints(output_sent),getSentimentPoints(input_sent)))
            
    def calculateSlots(self):
        self.overall_slot_list=[]
        for index in range(self.round_counter):
            output_sent = self.round[index][0]
            input_sent = self.round[index][1]
            output_result = nlu(output_sent,tokenizer, joint_model, intent_names, slot_names)
            input_result = nlu(output_sent,tokenizer, joint_model, intent_names, slot_names)
            mergedDict = mergeDict(output_result["slots"],input_result["slots"])
            if index == 0 :
                self.overall_slot_list.append(mergedDict)
            else :
                oldDict = self.overall_slot_list[index-1]
                currentDict = mergeDict(oldDict,mergedDict)
                self.overall_slot_list.append(currentDict.copy())

                               
               
    def avgSentiment(self,index):
        if (index ==0):
            return 0,0
        else:
            inputList=[]
            outputList=[]
            for x,y in self.sentiment_list[0:index]:
                outputList.append(x)
                inputList.append(y)
            return sum(outputList)/len(outputList),sum(inputList)/len(inputList)
        
        
        
    def getSetiments(self,index):
        output_sentiment,input_sentiment = self.sentiment_list[index]
        output_sentiment_1 = 0
        input_sentiment_1 = 0
        output_sentiment_2 = 0
        input_sentiment_2 = 0
        if (index>0):
            output_sentiment_1,input_sentiment_1 = self.sentiment_list[index-1]
        if (index>1):
            output_sentiment_2,input_sentiment_2 = self.sentiment_list[index-2]           
        
        return output_sentiment,input_sentiment,output_sentiment_1,input_sentiment_1,output_sentiment_2,input_sentiment_2 
    
    def getSlotNumbers(self,index):
        extraSlotNum = 0
        extraSlotNum_1 = 0
        extraSlotNum_2 = 0 
        currentDict = self.overall_slot_list[index]
        if(index>0):
            historyDict = self.overall_slot_list[index-1]
            extraSlotNum = countDictValue(currentDict) - countDictValue(historyDict)
        if(index>1):
            historyDict_1 = self.overall_slot_list[index-2]
            extraSlotNum_1 = countDictValue(historyDict) - countDictValue(historyDict_1)    
        if(index>2):
            historyDict_2 = self.overall_slot_list[index-3]
            extraSlotNum_2 = countDictValue(historyDict_1) - countDictValue(historyDict_2)         
        return extraSlotNum,extraSlotNum_1,extraSlotNum_2
    
    
    

                               
    # return points later on for SVM/OCNN      
    # return current output/ionput motion points,props,slots number for input/output,and its props.
    def getMetrix(self,index):
        identity = self.id + "|" + str(index)
        if index > self.round_counter:
            print("failed:",index)
            return None
        else:
            output_avg_sentiment,input_avg_sentiment=self.avgSentiment(index)
            currentDict = self.overall_slot_list[index]
            output_sentiment,input_sentiment,output_sentiment_1,input_sentiment_1,output_sentiment_2,input_sentiment_2 = self.getSetiments(index)
            extraSlotNum,extraSlotNum_1,extraSlotNum_2 = self.getSlotNumbers(index)
            input_similar_1,output_similar_1,input_similar_2,output_similar_2 = self.getSimilarity(index)
            # index is the actual index-1 time size of the history
            avgSlotNum = countDictValue(currentDict)/(index+1)
            return [identity,output_sentiment,input_sentiment,output_sentiment_1,input_sentiment_1,output_sentiment_2,input_sentiment_2,output_avg_sentiment,input_avg_sentiment,extraSlotNum,extraSlotNum_1,extraSlotNum_2,avgSlotNum,input_similar_1,output_similar_1,input_similar_2,output_similar_2]
    
     






for index, row in dst2data.iterrows():
    if(index==0):
        record = dialog(row["id"],row["input"],row["output"])
        if record.round_counter<2 :
            pass
        else :
            a = datetime.datetime.now()
            for index in range(1,record.round_counter):
                output = record.getMetrix(index)
                print(output)

            b = datetime.datetime.now()
            print(b-a)
            
    

    

        
        

In [ ]:
a = datetime.datetime.now()
getSentimentPoints("hello world")  
b = datetime.datetime.now()
print(b-a)

In [ ]:
a = datetime.datetime.now()
embed(["hello world"])  
nlu('tokyo restaurant in the north of town',
    tokenizer, joint_model, intent_names, slot_names)
b = datetime.datetime.now()
print(b-a)

In [ ]:
'''

import csv
from IPython.display import FileLink

f = open('generated.csv', 'w',newline='', encoding='utf-8')
writer = csv.writer(f)
header = ["id","output-sentiment","input-sentiment","output_sentiment_1","input_sentiment_1","output_sentiment_2","input_sentiment_2","output_avg_sentiment","input_avg_sentiment","slot_in_round","slot_in_round_1","slot_in_round_2","avg_slots","input_similar_1","output_similar_1","input_similar_2","output_similar_2"]
writer.writerow(header)
count = 0
for no,row in dst2data.iterrows():
    try:
        record = dialog(row["id"],row["input"],row["output"])
        # we dont care about the first round, start from second round
        if record.round_counter <2 :
            pass
        elif no in [45,52,50,51,102,146,192,226,229,261,289,298,360,388,416,423,469,501,507]:
            print("ignore id ",row["id"])
        else :
            for index in range(1,record.round_counter):
                output = record.getMetrix(index)
                writer.writerow(output)
            count+=1
            if count>=200:
                print("finished")
                break
    except:
        print(row["id"])
            
            
f.close()
    


In [ ]:
'''
f = open('generated_val.csv', 'w',newline='', encoding='utf-8')
writer = csv.writer(f)
header = ["id","output-sentiment","input-sentiment","output_avg_sentiment","input_avg_sentiment","slot_in_round","avg_slots"]
writer.writerow(header)
count = 0
for no,row in dst2data.iterrows():
    try:
        record = dialog(row["id"],row["input"],row["output"])
        # we dont care about the first round, start from second round
        if record.round_counter <2 :
            pass
        elif no<=115:
            pass
        elif no in [45,52,50,51,102,146,192,226,229,261,289,298,360,388,416,423,469,501,507]:
            print("ignore id ",row["id"])
        else :
            for index in range(1,record.round_counter):
                output = record.getMetrix(index)
                writer.writerow(output)
            count+=1
            if count>=30:
                print("finished")
                break
    except:
        print(row["id"])
            
            
f.close()
'''

In [ ]:
'''
import csv
from IPython.display import FileLink

testdata = pd.read_csv("../input/modified-dst2/selected_failed_call.csv")


f = open('generated_test.csv', 'w',newline='', encoding='utf-8')
writer = csv.writer(f)
header = ["id","output-sentiment","input-sentiment","output_sentiment_1","input_sentiment_1","output_sentiment_2","input_sentiment_2","output_avg_sentiment","input_avg_sentiment","slot_in_round","slot_in_round_1","slot_in_round_2","avg_slots","input_similar_1","output_similar_1","input_similar_2","output_similar_2"]
writer.writerow(header)
for _, row in testdata.iterrows():
    try:
        record = dialog(row["id"],row["input"],row["output"])
        output = record.getMetrix(record.round_counter-1)
        writer.writerow(output)
    except:
        print(row["id"])
            
            
f.close()

In [ ]:
'''
import csv
from IPython.display import FileLink

testdata = pd.read_csv("../input/failedcall/failedcall_removed.csv")


f = open('generated_test_extra.csv', 'w',newline='', encoding='utf-8')
writer = csv.writer(f)
header = ["id","output-sentiment","input-sentiment","output_sentiment_1","input_sentiment_1","output_sentiment_2","input_sentiment_2","output_avg_sentiment","input_avg_sentiment","slot_in_round","slot_in_round_1","slot_in_round_2","avg_slots","input_similar_1","output_similar_1","input_similar_2","output_similar_2"]
writer.writerow(header)
for _, row in testdata.iterrows():
    try:
        record = dialog(row["id"],row["input"],row["output"])
        output = record.getMetrix(record.round_counter-1)
        writer.writerow(output)
    except:
        print(row["id"])
            
            
f.close()